In [1]:
!pip install --upgrade biopython nglview ipywidgets

In [2]:
from Bio.PDB import *
import nglview as nv
import ipywidgets

In [4]:
cif_parser = MMCIFParser()
structure = cif_parser.get_structure("6EBK", "/home/onyxia/work/TDA_Protein3D/data/AF-P04637-F1-model_v4.cif")
#pdb_parser = PDBParser(QUIET=True)
#structure = pdb_parser.get_structure("6EBK", "/home/onyxia/work/TDA_Protein3D/notebooks/test.pdb")
view = nv.show_biopython(structure)

In [5]:
view

NGLWidget()